In [1]:
import pandas as pd

df = pd.read_excel('hate_crime_dataset.xlsx')
df.head()

,data_year,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,total_offender_count,offender_race,victim_count,offense_name,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,2017,AK,Alaska,Pacific,West,4,"Cities from 25,000 thru 49,999",2017-12-22,1,Others,1,Simple Assault,Residence/Home,Anti-White,Individual,S,S
1,2017,AK,Alaska,Pacific,West,4,"Cities from 25,000 thru 49,999",2017-10-24,0,Unknown,1,Destruction/Damage/Vandalism of Property,School-College/University,"Anti-Multiple Races, Group",Government,S,S
2,2017,AK,Alaska,Pacific,West,8A,"Non-MSA counties 100,000 or over",2017-05-04,0,Unknown,1,Destruction/Damage/Vandalism of Property,Church/Synagogue/Temple/Mosque,Anti-Other Christian,Religious Organization,S,S
3,2017,AK,Alaska,Pacific,West,1C,"Cities from 250,000 thru 499,999",2017-10-29,1,Black or African American,1,Aggravated Assault,Amusement Park,Anti-White,Individual,S,S
4,2017,AL,Alabama,East South Central,South,7,"Cities under 2,500",2017-09-23,1,Black or African American,1,Intimidation,School-College/University,Anti-White,Individual,S,S


In [2]:
pop_2017 = pd.read_csv('dataset/pop_df_2017.csv')
pop_2018 = pd.read_csv('dataset/pop_df_2018.csv')
pop_2019 = pd.read_csv('dataset/pop_df_2019.csv')
pop_2020 = pd.read_csv('dataset/pop_df_2020.csv')
pop_2021 = pd.read_csv('dataset/pop_df_2021.csv')

# Convert nnn,nnn,nnn(type: object) into nnnnnnnnn(type: int64) 
for i in range(len(pop_2021)):
    pop_2021.loc[i, 'Population'] = pop_2021['Population'].values[i].replace(",", "")
pop_2021 = pop_2021.astype({'Population':'int'})

pop_2021.head()

,Description,Population
0,Alabama,5049846
1,Alaska,734182
2,Arizona,7264877
3,Arkansas,3028122
4,California,39142991


In [3]:
race_portion = pd.read_csv('dataset/race_portion.csv')

races = list(race_portion.columns)
races.remove('State')

for race in races:
    race_portion[race] = race_portion[race].str.rstrip('%').astype('float') / 100.0

race_portion.head()

,State,White,Black,Hispanic,Asian,Multiple,Others
0,Alabama,0.65,0.27,0.04,0.01,0.02,0.00
1,Alaska,0.60,0.02,0.07,0.06,0.08,0.17
2,Arizona,0.54,0.04,0.32,0.03,0.02,0.04
3,Arkansas,0.72,0.15,0.08,0.02,0.02,0.01
4,California,0.36,0.05,0.40,0.15,0.03,0.00


In [4]:
def raceCalc(df_pop):
    df = df_pop.copy()
    for race in races:
        df[race] = df['Population'] * race_portion[race]
        df[race] = df[race].astype('int64')
    df.rename(columns={'Description':'State'}, inplace=True)
    df = df[['State']+races+['Population']]
    return df

In [5]:
pop_race_2017 = raceCalc(pop_2017)
pop_race_2018 = raceCalc(pop_2018)
pop_race_2019 = raceCalc(pop_2019)
pop_race_2020 = raceCalc(pop_2020)
pop_race_2021 = raceCalc(pop_2021)

In [6]:
df_2017 = df.loc[(df['data_year'] == 2017)]
df_2018 = df.loc[(df['data_year'] == 2018)]
df_2019 = df.loc[(df['data_year'] == 2019)]
df_2020 = df.loc[(df['data_year'] == 2020)]
df_2021 = df.loc[(df['data_year'] == 2021)]

cntcols = ['Year', 'State'] + races
df_count = pd.DataFrame(columns=cntcols)

def count_bias(series):
    for race in races:
        row[race] = df.loc[df['bias_desc'].str.contains(race)].shape[0]
    df_count.loc[year] = row
    print(row)
    print(df_count)

In [7]:
#Anti-Black
def get_ab_count(series):
    res = len([_ for x in series if 'Black' in x])
    return res

#Anti-Hispanic
def get_ah_count(series):
    res = len([_ for x in series if 'Hispanic' in x])
    return res

#Anti-Asian
def get_aa_count(series):
    res = len([_ for x in series if 'Asian' in x])
    return res

#Anti-Multiple
def get_am_count(series):
    res = len([_ for x in series if 'Multiple' in x])
    return res

#Anti-Others
def get_ao_count(series):
    res = len([_ for x in series if 'Others' in x])
    return res

# def get_anti_count(series):
#     res = list()
#     for race in races:
#         res.append(len([_ for x in series if race in x]))
#     return res

def count_bias(df):
    df_tmp = df.groupby('state_name').agg({'bias_desc':[get_ab_count, get_ah_count, get_aa_count, get_am_count, get_ao_count]}).reset_index()
    df_tmp.columns = ['State', 'Black', 'Hispanic', 'Asian', 'Multiple', 'Others']
    df_tmp.drop(df_tmp[df_tmp['State']=='Federal'].index, inplace=True)
    df_tmp.sort_values(by='State', inplace=True)
    df_tmp.reset_index(drop=True, inplace=True)
    
    return df_tmp
    
# df_2017_bias = df_2017.groupby('state_name').agg({'bias_desc':[get_ab_count, get_ah_count, get_aa_count, get_am_count, get_ao_count]}).reset_index()
# df_2017.groupby('state_name').agg({'bias_desc':get_anti_count}).reset_index()

In [8]:
df_2017_bias = count_bias(df_2017)
df_2018_bias = count_bias(df_2018)
df_2019_bias = count_bias(df_2019)
df_2020_bias = count_bias(df_2020)
df_2021_bias = count_bias(df_2021)

,State,Black,Hispanic,Asian,Multiple,Others
0,Alabama,65,3,1,7,0
1,Alaska,2,0,0,0,0
2,Arizona,26,8,1,3,0
3,Arkansas,13,7,1,0,0
4,California,23,9,9,0,0
5,Colorado,74,31,14,5,0
6,Connecticut,35,8,6,1,0
7,Delaware,3,0,1,0,0
8,District of Columbia,12,3,3,0,0
9,Florida,1,0,0,0,0


In [9]:
def normalize(df, pop):
    df_tmp = df.copy()
    for race in races:
        if race=='White':
            pass
        else:
            df_tmp[race] = df_tmp[race] * 10000 / pop[race]
    return df_tmp
    
norm_2017_race = normalize(df_2017_bias, pop_race_2017)
norm_2018_race = normalize(df_2018_bias, pop_race_2018)
norm_2019_race = normalize(df_2019_bias, pop_race_2019)
norm_2020_race = normalize(df_2020_bias, pop_race_2020)
norm_2021_race = normalize(df_2021_bias, pop_race_2021)

,State,Black,Hispanic,Asian,Multiple,Others
0,Alabama,0.476729,0.148520,0.198028,0.693097,NaN
1,Alaska,1.362119,0.000000,0.000000,0.000000,0.0
2,Arizona,0.894716,0.034412,0.045883,0.206474,0.0
3,Arkansas,0.286206,0.288959,0.165120,0.000000,0.0
4,California,0.117518,0.005748,0.015328,0.000000,NaN
5,Colorado,3.183467,0.242474,0.803038,0.286799,0.0
6,Connecticut,0.965957,0.129876,0.331186,0.091996,NaN
7,Delaware,0.135712,0.000000,0.248806,0.000000,NaN
8,District of Columbia,0.398731,0.407792,1.121453,0.000000,NaN
9,Florida,0.003054,0.000000,0.000000,0.000000,NaN
